# Exercise 14 Balloon  Experiment

#### In an experiment to measure the flux of cosmic rays in the upper atmosphere, protons with an energy between 1 GeV and 100 GeV are counted over a period of one hour from a flying balloon. Over a period of one week, a measurement run of one hour duration is made every day. The measured data are:

#### a) Assume that the cosmic ray flux is constant over the measurement period. Calculate the most probable count rate using the maximum likelihood method.

We can use a possion-distribution to describe our measurement since this is a counting experiment.
So we get our likelihood function $L(\lambda,i) = \prod\limits_{i}\left( \frac{\lambda^{x_i}}{x_i!}e^{-\lambda} \right)$
We have a $\lambda =$ const ,since the counting rate is constant and $\lambda$ describes exactly that.
To get the most propable count rate we minimize the negativ log likelihood function for $\lambda$.
$\frac{\partial}{\partial \lambda} (-1)\left( \ln\left( \prod\limits_{i}\left( \frac{\lambda^{x_i}}{x_i!}e^{-\lambda} \right) \right) \right) = 0$

$\leftrightarrow \frac{\partial}{\partial \lambda} (-1)\left( \sum\limits_{i} \ln\left( \frac{\lambda^{x_i}}{x_i!}e^{-\lambda} \right) \right) = 0$

$\leftrightarrow \frac{\partial}{\partial \lambda} (-1)\left( \sum\limits_{i} \left( x_i\ln(\lambda) - \ln(x_i!)-\lambda\right)\right) = 0$ 



$\leftrightarrow (-1)\sum\limits_{i} \left(\frac{x_i}{\lambda}-1\right) = (-1)\frac{1}{\lambda}\left( \sum\limits_{i} x_i\right)+N$ = 0


$\leftrightarrow \lambda = \frac{1}{N}\sum\limits_{i} x_i$

This is the most popable count rate.

#### b) Your colleague looks at the readings and hypothesizes that the cosmic ray flux is experiencing a dramatic increase. Assume a linearly increasing flux and calculate numerically the most probable flux parameters using the maximum likelihood method.

Now we assume $\lambda$ has a linear increase. So we have $\lambda_i = a\times i + \lambda_0$
With this we write down the negativ log-likelihood function as:
$F = -\ln(L) = -\ln \left( \prod\limits_{i}\left( \frac{(ai+\lambda_0)^{x_i}}{x_i!}e^{-ai-\lambda_0} \right) \right) = \sum\limits_{i} \left(-x_i\ln(ai+\lambda_0) + \ln(x_i!)+ ai+\lambda_0\right)$

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as op

#numerical calculation of the most propable flux parameters
ii = np.array([1,2,3,4,5,6,7])
x_i = np.array([4135,4202,4203,4218,4227,4231,4310])
def linearlambda(i,params):
    return params[0]*i+params[1]
def F_i(x,i,params):
    return -x*np.log(linearlambda(i,params))+linearlambda(i,params)
def F(params):
    return F_i(x_i[0],ii[0],params)+F_i(x_i[1],ii[1],params)+F_i(x_i[2],ii[2],params)+F_i(x_i[3],ii[3],params)+F_i(x_i[4],ii[4],params)+F_i(x_i[5],ii[5],params)+F_i(x_i[6],ii[6],params)
def lambda_a(x):
    return np.mean(x)
def F_i_new(count,params):
    return F_i(x_new[count],ii_new[count],params)
def F_new(params):
    return F_i_new(1,params)+F_i_new(2,params)+F_i_new(3,params)+F_i_new(4,params)+F_i_new(5,params)+F_i_new(6,params)+F_i_new(7,params)

parameters = op.minimize(F,[x_i[2]-x_i[1],x_i[1]], bounds=[(0,None),(0,None)])
print("The optimal Parameters are:")
print("a =",parameters.x[0])
print("lambda_0 =",parameters.x[1])


#calculation of the most probable lambda in a) for c)
print("Most propable lambda_a =",lambda_a(x_i))
lambda_av = lambda_a(x_i)

The optimal Parameters are:
a = 20.57199186555189
lambda_0 = 4137.118883371033
Most propable lambda_a = 4218.0


#### c) Calculate the significance of his observation using a likelihood ratio test. Evaluate the significance achieved. Hint: Assume that Wilks’ theorem is valid here. Why can you assume this?

With the two likelihood functions from before we can calculate the ratio $\Gamma$.
$\Gamma(i) = \frac{\prod\limits_{i}\left( \frac{(ai+\lambda_0)^{x_i}}{x_i!}e^{-ai-\lambda_0}\right)}{\prod\limits_{i}\left( \frac{\lambda^{x_i}}{x_i!}e^{-\lambda}\right)} = \prod\limits_{i}\left( \left( \frac{ai+\lambda_0}{\lambda} \right)^{x_i}e^{\lambda-ai-\lambda_0} \right)$
With Wilks Theorem we get:
$-2\ln(\Gamma) = \sum\limits_{i} \left(-2\left( x_i\ln(\frac{ai+\lambda_0}{\lambda}) +\lambda - ai -\lambda_0\right)\right)$

In [52]:
#using the calculated parameters from before:
params_1 = np.array([lambda_av,parameters.x[0],parameters.x[1]])
def ratio_wilks_i(x,i,params_r):
    return -2*(x*np.log((params_r[1]*i+params_r[2])/params_r[0])+params_r[0]-params_r[1]*i-params_r[2])
ratio = 0
for i in range(len(ii)):
    ratio = ratio + ratio_wilks_i(x_i[i],ii[i],params_1)
print("The Wilks-Ratio(or whatever) is given by Gamma_W =",-ratio)
print("the Ratio is given by Gamma=",np.exp(-2*ratio))

The Wilks-Ratio(or whatever) is given by Gamma_W = 3.106839501484501
the Ratio is given by Gamma= 499.53566801117654


It is valid to use Wilks Theorem, because the Nullhypothesis($\lambda =$ linear) is a linear transfrmation of the alternative hypothesis($\lambda =$const) and the sample size is big enough.

### d) Your colleague performs another measurement a week later to support his thesis. His measurement results in Day 14 Counts 4402 Calculate (a) to (c) again for this new data set.

In [53]:
#initialisation of the new dataset
x_new = np.array([4135,4202,4203,4218,4227,4231,4310,4402])
ii_new = np.array([1,2,3,4,5,6,7,14])

lambda_a_new = lambda_a(x_new)
parameters_new = op.minimize(F_new,[x_new[2]-x_new[1],x_new[1]], bounds=[(0,None),(0,None)])
print("The most propable const lambda =",lambda_a_new)
print("The new optimal Parameters are:")
print("a_new =",parameters_new.x[0])
print("lambda_0_new =",parameters_new.x[1])

#ratio

params_1_new = np.array([lambda_a_new,parameters_new.x[0],parameters_new.x[1]])
ratio_new = 0
for i in range(len(ii_new)):
    ratio_new = ratio_new + ratio_wilks_i(x_new[i],ii_new[i],params_1_new)
print("The Wilks-Ratio(or whatever) is given by Gamma_W =",-ratio_new)
print("the Ratio is given by Gamma=",np.exp(-2*ratio_new))

The most propable const lambda = 4241.0
The new optimal Parameters are:
a_new = 17.944413991478562
lambda_0_new = 4150.536280945702
The Wilks-Ratio(or whatever) is given by Gamma_W = 9.905151403138007
the Ratio is given by Gamma= 401333297.6592493


#### e) What is the methodological problem with exercise d)’s approach? Why should you not publish these results, even if the significance is higher then some preset threshold (e.g. 3 or 5𝜎)?

The new added value has a big time gap between the measurement. So The new data set is not able to describe the phenomenon from day 7 to day 13.